# Lab 7

In [1]:
# %pip install geemap
%pip install geedim

In [2]:
# Import packages
import ee
import geemap

In [3]:
geemap.ee_initialize()

In [4]:
# Check initialize
Map = geemap.Map()
# Map

# Question 1

Create a fishnet with a 4-degree interval based on the extent of [-112.5439, 34.0891, -85.0342, 49.6858]. Use the fishnet to download the Landsat 7 image tiles by the fishnet using the geemap.download_ee_image_tiles() function. Relevant Earth Engine assets:

ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')


In [5]:
Map1 = geemap.Map()

# Set ROI
roi = ee.Geometry.BBox(-112.5439, 34.0891, -85.0342, 49.6858)
#Map1.addLayer(roi, {}, 'ROI')

# Create fishnet with 4-degree interval
fishnet = geemap.fishnet(roi, h_interval=4.0, v_interval=4.0)
style = {'color': 'blue', 'fillColor': '00000000'}

# Load Landsat Image
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B4', 'B3', 'B2'])

# Add layers

trueColor321Vis = {
  'min': 0,
  'max': 120,
  'gamma': 1.3
};

Map1.addLayer(image, trueColor321Vis, 'Landsat')
Map1.addLayer(fishnet.style(**style), {}, 'Fishnet')

# Add text to the map
text = "Created by Pablo Miranda"
Map1.add_text(text, fontsize=20, position='bottomright')

# Download using fishnet (28 tiles) -> stored in Files folder
# geemap.download_ee_image_tiles(image, fishnet, crs="EPSG:3857", scale=30)

Map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/WXY9lbe.jpeg)



![](https://i.imgur.com/Nwa2MVM.jpeg)

# Question 2

Create annual cloud-free Landsat imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer.

In [6]:
Map2 = geemap.Map()

# Load Val Verde countie
borders = ee.FeatureCollection("TIGER/2018/Counties")
valverde = borders.filter(ee.Filter.eq('NAME', 'Val Verde'))

roi = valverde.first().get('Geometry')

# Create list sequence in gee
years = ee.List.sequence(2017, 2023)

# Create function for each image
def yearly_image(year):

    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, "year")

    collection = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1')
        .filterDate(start_date, end_date)
        .filterBounds(valverde)
    )

    # Use Landsat simpleComposite for free cloud cover image
    image = ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(valverde)

    return image

# Map function passing year list
images = years.map(yearly_image)

# Add cloud-free Landsat layers
vis_params = {'bands': ['B5', 'B4', 'B3'], 'max': 128}

# Adding and downloading images with a for loop
for index in range(0, 7):
    image = ee.Image(images.get(index))
    layer_name = "Year " + str(index + 2017) + ".tif"
    Map2.addLayer(image, vis_params, layer_name)

    # Downloading each image -> stored in Files folder
    #geemap.download_ee_image(image, filename= layer_name, scale=10000,crs="EPSG:3857", region = roi) # 10km resolution in order to reduce time

# Add Valverde border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}
Map2.addLayer(valverde.style(**style), {}, "Val Verde")
Map2.centerObject(valverde, 9)

# Add text to the map
text = "Created by Pablo Miranda"
Map2.add_text(text, fontsize=20, position='bottomright')
Map2

Map(center=[29.892652714069406, -101.1511609925234], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/FIs5oPf.jpeg)

![](https://i.imgur.com/dABkRow.jpeg)

# Question 3

Create annual cloud-free Sentinel-2 imagery for the years 2017-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process. Narrow down the date range (e.g., summer months) to reduce the number of images, which can avoid memory errors.



In [7]:
Map3 = geemap.Map()

# Load Val Verde countie
borders = ee.FeatureCollection("TIGER/2018/Counties")
valverde = borders.filter(ee.Filter.eq('NAME', 'Val Verde'))

roi = valverde.first().get('Geometry')

# Load Landsat 8 Images
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterMetadata(
    'CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)
    .filter(ee.Filter.calendarRange(6, 8, 'month')) # Only summer months
    )

start_date = '2017-01-01'
end_date = '2024-01-01'

ts = geemap.create_timeseries(collection, start_date, end_date, valverde,
                               frequency = "year", reducer = 'median')
ts

In [8]:
Map3 = geemap.Map()

vis_params_left = {"min": 0, "max": 4000, "bands": ["B8", "B4", "B3"]}
vis_params_right = {"min": 0, "max": 4000, "bands": ["B3", "B2", "B1"]}

years = ts.aggregate_array('system:index').getInfo()
labels = [str(y) for y in range(2017, 2024)]

# Add timeseries inspector
Map3.ts_inspector(
    left_ts=ts,
    right_ts=ts,
    left_names=labels,
    right_names=labels,
    left_vis=vis_params_left,
    right_vis=vis_params_right,
    width='80px',
)

# Add Valverde border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}
Map3.addLayer(valverde.style(**style), {}, "Val Verde")
Map3.centerObject(valverde, 9)

# Add text to the map
text = "Created by Pablo Miranda"
Map3.add_text(text, fontsize=20, position='bottomright')

Map3

Map(center=[29.892652714069406, -101.1511609925234], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
# Alternative download method -> Exporting image collections
out_dir = 'Downloads'
geemap.ee_export_image_collection(ts, out_dir=out_dir, scale=90000, region = roi)

Total number of images: 7

Exporting 1/7: Downloads/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/0.tif


Exporting 2/7: Downloads/1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/1.tif


Exporting 3/7: Downloads/2.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/2.tif


Exporting 4/7: Downloads/3.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/3.tif


Exporting 5/7: Downloads/4.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/4.tif


Exporting 6/7: Downloads/5.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/5.tif


Exporting 7/7: Downloads/6.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads/6.tif




![](https://i.imgur.com/dPpWh2b.jpeg)



![](https://i.imgur.com/dq8nZic.jpeg)

# Question 4

Create annual cloud-free NAIP imagery for the years 2010-2023 for a US county of your choice. Download the images to your computer. You can download a coarse resolution image to speed up the download process.

In [10]:
Map4 = geemap.Map()

# Load Val Verde countie
borders = ee.FeatureCollection("TIGER/2018/Counties")
valverde = borders.filter(ee.Filter.eq('NAME', 'Val Verde'))

roi = valverde.first().get('Geometry')

# Load naip collection
collection_naip = geemap.naip_timeseries(valverde, start_year=2010, end_year=2024, RGBN=True)
years = geemap.image_dates(collection_naip, date_format='YYYY').getInfo()

# Add layers
size = len(years)
images = collection_naip.toList(size)
for i in range(size):
    image = ee.Image(images.get(i))
    Map4.addLayer(image, {'bands': ['R', 'G', 'B']}, years[i])

# Add Valverde border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}
Map4.addLayer(valverde.style(**style), {}, "Val Verde")
Map4.centerObject(valverde, 9)

# Add text to the map
text = "Created by Pablo Miranda"
Map4.add_text(text, fontsize=20, position='bottomright')

Map4

Map(center=[29.892652714069406, -101.1511609925234], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
# Download naip collection
out_dir = 'Downloads_naip'
geemap.ee_export_image_collection(collection_naip, out_dir=out_dir, scale=90000, region = roi)

Total number of images: 8

Exporting 1/8: Downloads_naip/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/0.tif


Exporting 2/8: Downloads_naip/2.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/2.tif


Exporting 3/8: Downloads_naip/4.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/4.tif


Exporting 4/8: Downloads_naip/6.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/6.tif


Exporting 5/8: Downloads_naip/8.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/8.tif


Exporting 6/8: Downloads_naip/9.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/9.tif


Exporting 7/8: Downloads_naip/10.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/10.tif


Exporting 8/8: Downloads_naip/12.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Downloads_naip/12

![](https://i.imgur.com/8cNvaUd.jpeg)


![](https://i.imgur.com/Tqzcf5D.jpeg)

# Question 5
Download a US county of your choice and save it as a shapefile or GeoJSON file.


In [12]:
Map5 = geemap.Map()

# Load Val Verde countie
borders = ee.FeatureCollection("TIGER/2018/Counties")
valverde = borders.filter(ee.Filter.eq('NAME', 'Val Verde'))

# Add Valverde border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}
Map5.addLayer(valverde.style(**style), {}, "Val Verde")
Map5.centerObject(valverde, 9)

# Download as geojson
geemap.ee_to_geojson(valverde, filename='valverde.geojson')

# Download as shp
geemap.ee_to_shp(valverde, filename='valeverde.shp')

Map5

Map(center=[29.892652714069406, -101.1511609925234], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/duQoznt.jpeg)

# ¡¡THANK YOU FOR THIS AMAZING COURSE!!!
